Imports:

In [8]:
# IMPORTS
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.widgets import Button
%matplotlib qt
from train_for_real import *
import pickle
from model_settings import get_args
from finetuneSAM.models.sam_LoRa import LoRA_Sam

In [25]:
# LOAD MODEL
setting = 6

args =  get_args(setting)
sam = load_model(args)

if args.finetune_type == 'lora':
    sam = LoRA_Sam(args,sam,r=2).sam
    

# checkpoints\eLdL\+checkpoint_best.pth
# with open('checkpoints\eLdL\+checkpoint_best.pth', 'rb') as file:
#     model_checkpoint = torch.load(file)

# sam.load_state_dict(model_checkpoint)
sam.to('cuda')
sammy = Sammy(sam, (512,512))


In [26]:
# LOAD DATA
with open('small_data_5.pkl', 'rb') as file:
    small_data_5 = pickle.load(file)
small_data_5 = EyeData(small_data_5)
small_data_5.divide_into_batches(1,augment=False)

In [27]:


mini_b = small_data_5.batches[1]

imgs,ps,lbls, masks = mini_b['image'],mini_b['points'],mini_b['p_labels'],mini_b['mask']

sammy.set_images(imgs)

ret = sammy.predict_from_set(None,None,masks, visualize=True)

current_image = ret['visual'][0]


# target_points = []
# background_points = []
##print("Points: ",ps)
ps = ps *0.5
#print("Points: ",ps)
# label == 1
target_points = ps[0][lbls[0] == 1]
##print("Target points: ",target_points)
background_points = ps[0][lbls[0] == 0]
#print("Background points: ",background_points)

target_points_to_add = []
background_points_to_add = []

point_label = None
running = True


def scatter_ps(img, points, color):
    # draw points 5x5
    #print("Points: ",points)
    for point in points:
        from_x = point[0]-2
        to_x = point[0]+2
        from_y = point[1]-2
        to_y = point[1]+2

        from_x = max(0,from_x)
        to_x = min(img.shape[0],point[0]+2)
        from_y = max(0,point[1]-2)
        to_y = min(img.shape[1],point[1]+2)
        img[from_x:to_x,from_y:to_y] = color

    return img

def revisualize(new_target_points:np.array, new_background_points:np.array):
    
    global current_image
    global target_points
    global background_points
    non_empty = []

    if len(target_points) != 0:
        non_empty.append(target_points)
    
    if len(new_target_points) != 0:
        non_empty.append(new_target_points)

    if len(background_points) != 0:
        non_empty.append(background_points)
    if len(new_background_points) != 0:
        non_empty.append(new_background_points)
    
    if non_empty == []:
        return sammy.predict_from_set(None,None,masks, visualize=True)['visual'][0]

    all_points = np.concatenate(non_empty,axis=0)
    cnt_target_points = len(target_points) + len(new_target_points)

    target_points = all_points[:cnt_target_points]
    background_points = all_points[cnt_target_points:]

    all_labels = np.zeros(len(all_points))
    all_labels[:cnt_target_points] = 1
    
    #reshape
    all_points = all_points.reshape((1,all_points.shape[0],2)).astype(np.int32)
    all_labels = all_labels.reshape((1,all_labels.shape[0]))

    # measure time 
    #import time
    #start = time.time()
    
    _ret = sammy.predict_from_set((all_points*2).astype(np.int32),all_labels,masks, visualize=True, score=True)
    #print("Time for prediction: ",time.time()-start)

    # 255, g: 192, b: 203
    img = _ret['visual'][0]
    if cnt_target_points != 0:
        img = scatter_ps(img,all_points[0][:cnt_target_points],[255,192,200])
    #print("Coloring background points: ")
    if len(background_points) != 0:
        img = scatter_ps(img,all_points[0][cnt_target_points:],[0,0,200])
    #current_image = _ret['visual'][0]

    print("Dice: ",_ret['loss_dice'], "BCE: ",_ret['loss_bce'])
    
    
    return img

# revisualize(np.array([[50,50],[100,100]]),
#             np.array([[70,70],[130,130]]))

#revisualize([],[])

i = imgs[0]
# aadd alpha channel
#i = np.concatenate([i,np.ones((512,512,1))],axis=2)
plt.imshow(i)
# make it half transparent
#i[:,:,3] = 0.5


#plt.imshow(i)



   

In [28]:
#def main():
global current_image
global current_score
#plt.axis([-20,20,0,10000])
fig, axs = plt.subplots(nrows=4,ncols=1, width_ratios=[1],height_ratios=[0.9,0.040,0.040,0.02] ,figsize=(7, 9))

ax1 = axs[0]
ax2 = axs[1]
ax3 = axs[2]
ax4 = axs[3]

#if event.button == 3:

def on_left_click(x,y):
    global current_image
    if point_label is None:
        return
    elif point_label == 0:
        background_points_to_add.append([x,y])
    elif point_label == 1:
        target_points_to_add.append([x,y])
    current_image = revisualize(np.array(target_points_to_add),np.array(background_points_to_add))
    target_points_to_add.clear()
    background_points_to_add.clear()

    ax1.imshow(current_image,aspect='auto')
    #ax1.draw()
    #fig.draw()
    plt.draw()
    #plt.pause(0.001)
    #fig.canvas.flush_events()

def on_right_click(x,y):
    global background_points
    global target_points
    global current_image

    if point_label is None:
        return
    elif point_label == 0:
        # find the closest point in background_points
        min_dist = 300
        min_point = None
        indx_to_delete = 0
        for i in range(len(background_points)):
            point = background_points[i]
            # find the closest point in up to 5x5
            dist = np.linalg.norm(np.array(point)-np.array([x,y]))
            if dist < min_dist:
                min_dist = dist
                min_point = point
                indx_to_delete = i
        if min_point is not None and min_dist < 10:
            #print("Deleting point: ",min_point, " with distance: ",min_dist, " from: ",[x,y])
            background_points = np.delete(background_points,indx_to_delete,axis=0)
            if background_points.shape[0] == 0:
                print("Background points are empty")
                background_points = []

    elif point_label == 1:
        # find the closest point in target_points
        min_dist = 300
        min_point = None
        for i in range(len(target_points)):
            point = target_points[i]
            # find the closest point in up to 5x5
            dist = np.linalg.norm(np.array(point)-np.array([x,y]))
            if dist < min_dist:
                min_dist = dist
                min_point = point
        if min_point is not None and min_dist < 10:
            target_points = np.delete(target_points,i, axis=0)
            if target_points.shape[0] == 0:
                print("Target points are empty")
                target_points = []
    
    
    current_image = revisualize([],[])#(np.array(target_points),np.array(background_points))
    
    
    ax1.imshow(current_image,aspect='auto')
    plt.draw()
    #fig.canvas.flush_events()

def on_image_click(event):
    #print('The Click Event Triggered!:',event.xdata, event.ydata)
    if not event.inaxes == ax1:
        return
    y,x= int(event.xdata),int(event.ydata)
    if event.button == 1:
        #print('The Left Click Event Triggered!:',event.xdata, event.ydata)
        on_left_click(x,y)
    elif event.button == 3:
        #print('The Right Click Event Triggered!:',event.xdata, event.ydata)
        on_right_click(x,y)

    
def on_close(event):
    global running
    running = False
    plt.close('all')
    
cid = fig.canvas.mpl_connect('button_release_event', on_image_click)
_cid2 = fig.canvas.mpl_connect('close_event', on_close)

bg_button = Button(ax2, 'Background', color='white', hovercolor='gold')
t_button = Button(ax3, 'Target', color='white', hovercolor='gold')

def on_checkbox_click(event):
    global point_label
    if event.inaxes == ax2:
        #'Background'
        if bg_button.color == 'red':
            bg_button.color = 'white'
            point_label = None
        else:
            bg_button.color = 'red'
            point_label = 0
        t_button.color = 'white'
    elif event.inaxes == ax3:
        # Target
        if t_button.color == 'green':
            t_button.color = 'white'
            point_label = None
        else:
            t_button.color = 'green'
            point_label = 1
        bg_button.color = 'white'
    else:
        return
    
bg_button.on_clicked(on_checkbox_click)
t_button.on_clicked(on_checkbox_click)



current_image = revisualize([],[])
#axs[0].text(0.1, 0.5, 'Click me anywhere on this plot!', dict(size=20))
ax1.imshow(current_image,aspect='auto')
#ax1.axis('off')

plt.ion()
plt.show()
plt.draw()

while running:
    #pass
    # ax1.imshow(current_image,aspect='auto')
    # #ax1.draw()
    # #fig.draw()
    # plt.draw()
    # #plt.pause(0.001)
    fig.canvas.flush_events()


Dice:  tensor(0.7002, device='cuda:0') BCE:  tensor(1.0518, device='cuda:0')
Dice:  tensor(0.7007, device='cuda:0') BCE:  tensor(1.0113, device='cuda:0')
Dice:  tensor(0.7062, device='cuda:0') BCE:  tensor(1.0320, device='cuda:0')
Dice:  tensor(0.7050, device='cuda:0') BCE:  tensor(0.9513, device='cuda:0')
Dice:  tensor(0.7003, device='cuda:0') BCE:  tensor(0.8920, device='cuda:0')
Dice:  tensor(0.6947, device='cuda:0') BCE:  tensor(0.7862, device='cuda:0')
Dice:  tensor(0.6933, device='cuda:0') BCE:  tensor(0.7646, device='cuda:0')
Dice:  tensor(0.6918, device='cuda:0') BCE:  tensor(0.7544, device='cuda:0')
Dice:  tensor(0.6881, device='cuda:0') BCE:  tensor(0.7336, device='cuda:0')
Dice:  tensor(0.6867, device='cuda:0') BCE:  tensor(0.7086, device='cuda:0')
Dice:  tensor(0.6924, device='cuda:0') BCE:  tensor(0.7664, device='cuda:0')
Dice:  tensor(0.6962, device='cuda:0') BCE:  tensor(0.8165, device='cuda:0')
Dice:  tensor(0.6992, device='cuda:0') BCE:  tensor(0.8561, device='cuda:0')